<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=667e3a846e2dbdd2411d809b4f7a95d331d06d939646622eb3cdc13de52ef9e1
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-12 13:43:51
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.18 L
Current:  1.41 C
-------------------
Today PnL: -3.34 K (-0.02%)
Current PnL: -17.75 L (-11.94%)
CY Booked + Current PnL: -6.72 L (-4.52%)
-------------------
Total profit:  2.09 L
Total loss:  -19.84 L
-------------------
Total Booked + Current PnL: 20.26 L (16.65%)
Total Booked PnL: 38.01 L (31.23%)
Curr Year Booked PnL: 11.03 L (7.8%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 83.86 L (59.28%)
Deployed:  1.22 C
Current:  1.41 C
CAGR/XIRR %: 9.4%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,APOLLOHOSP,8285.00,-13.20,60.0,H-LC,11.19,173404.0,17227.0,8861.0,0.03,11.03,5.11,16.71,22.0,1.94,1.26,30.39,X40N,BTT,HEALTHCARE
62,SBIN,863.00,-12.30,60.0,M-LC,4.27,218135.0,16656.0,10558.0,-0.06,8.27,4.84,13.51,60.0,1.58,1.59,21.90,XY25,NTT,BANKS
77,TTKPRESTIG,770.00,103.84,58.0,M-SC,4.99,87849.0,-12928.0,13019.0,-0.38,-12.83,14.82,0.09,245.0,-0.99,0.64,15.41,OX40N,NTT,DURABLES
49,NATIONALUM,247.44,-42.83,71.0,H-MC,3.48,113632.0,13128.0,17511.0,2.52,13.06,15.41,30.49,79.0,0.75,0.83,49.70,MH,ATH,METALS
41,ITC,452.00,-37.08,56.0,H-LC,2.98,202837.0,2699.0,18194.0,-0.06,1.35,8.97,10.44,4.0,0.15,1.48,6.99,X40,NTT,FMCG


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,ICICIPRULI,790.00,-21.22,41.0,H-MC,2.08,135532.0,-256.0,43004.0,0.21,-0.19,31.73,31.48,107.0,-0.01,0.99,11.91,X40,ATH,INSURANCE
30,HAVELLS,2069.16,-0.94,57.0,H-MC,1.67,245606.0,-2125.0,77194.0,-0.06,-0.86,31.43,30.30,92.0,-0.03,1.79,12.32,X40,ATH,ELECTRICAL
12,BANKINDIA,190.00,-30.43,61.0,H-MC,5.62,180561.0,753.0,111659.0,-0.59,0.42,61.84,62.52,88.0,0.01,1.32,34.65,XR,NTT,BANKS


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-16.73,37.0,H-LC,6.05,299796.0,-16046.0,128702.0,-1.00,-5.08,42.93,35.67,5.0,-0.12,2.18,8.07,X40N,ATH,FMCG
43,JIOFIN,387.00,-1.64,43.0,H-LC,13.52,267374.0,9.0,65453.0,0.06,0.00,24.48,24.48,48.0,0.00,1.95,54.98,XY24,BTT,FINANCE
80,UNITDSPR,1644.00,-14.23,46.0,H-LC,8.06,224804.0,-9340.0,57954.0,-0.91,-3.99,25.78,20.77,86.0,-0.16,1.64,2.65,X40N,NTT,BREWERIES
31,HCLTECH,1943.91,0.65,48.0,H-LC,10.12,230068.0,-11856.0,75117.0,-0.16,-4.90,32.65,26.15,8.0,-0.16,1.68,8.72,X40,ATH,IT
56,RELIANCE,1533.00,-13.29,52.0,H-LC,6.04,217121.0,6755.0,22038.0,0.61,3.21,10.15,13.68,37.0,0.31,1.58,20.26,XY25,NTT,REFINERIES


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.2,-3.47,59.0,H-LC,1.19,239310.0,-6841.0,86056.0,0.50,-2.78,35.96,32.18,16.0,-0.08,1.74,32.26,X200,ATH,IT
41,ITC,452.0,-37.08,56.0,H-LC,2.98,202837.0,2699.0,18194.0,-0.06,1.35,8.97,10.44,4.0,0.15,1.48,6.99,X40,NTT,FMCG
50,NESTLEIND,1377.0,-7.33,62.0,H-LC,3.73,285012.0,19586.0,37194.0,-0.23,7.38,13.05,21.40,11.0,0.53,2.08,15.00,XY25,NTT,FMCG
20,CIPLA,1795.0,-19.44,55.0,H-LC,5.90,215309.0,10809.0,30595.0,0.69,5.29,14.21,20.25,10.0,0.35,1.57,14.48,X40N,BTT,PHARMA
56,RELIANCE,1533.0,-13.29,52.0,H-LC,6.04,217121.0,6755.0,22038.0,0.61,3.21,10.15,13.68,37.0,0.31,1.58,20.26,XY25,NTT,REFINERIES


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,HINDZINC,730.22,22.76,66.0,M-LC,1.71,199622.0,-5454.0,118017.0,2.83,-2.66,59.12,54.89,52.0,-0.05,1.45,20.79,X5K,ATH,METALS
49,NATIONALUM,247.44,-42.83,71.0,H-MC,3.48,113632.0,13128.0,17511.0,2.52,13.06,15.41,30.49,79.0,0.75,0.83,49.70,MH,ATH,METALS
75,TITAGARH,1548.00,-2.32,65.0,H-SC,5.82,160300.0,-31715.0,110607.0,1.68,-16.52,69.00,41.08,158.0,-0.29,1.17,35.83,XY24,NTT,ENGINEERING
58,ROUTE,2236.84,-46.55,48.0,H-SC,18.04,87870.0,-45196.0,138053.0,1.65,-33.97,157.11,69.78,118.0,-0.33,0.64,2.60,SR,ATH,IT
85,WHIRLPOOL,2270.00,-40.03,55.0,M-SC,1.41,101258.0,9760.0,68997.0,1.51,10.67,68.14,86.07,223.0,0.14,0.74,47.04,XR,NTT,DURABLES


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,ATULAUTO,844.0,3570.27,56.0,M-SC,7.49,114366.0,-33002.0,84814.0,-2.39,-22.39,74.16,35.16,236.0,-0.39,0.83,16.81,XY24,NTT,AUTO
53,RAJESHEXPO,518.0,1821.72,61.0,L-SC,5.56,52449.0,-84728.0,84821.0,-2.26,-61.77,161.72,0.07,267.0,-1.00,0.38,30.44,OX40N,NTT,JEWELLERY
64,SHALBY,327.0,1053.11,66.0,M-SC,2.00,156826.0,-24317.0,69788.0,-1.86,-13.42,44.50,25.10,235.0,-0.35,1.14,25.68,XY24,NTT,HEALTHCARE
69,SURYODAY,240.0,40.35,49.0,H-SC,3.27,128826.0,-50245.0,115969.0,-1.75,-28.06,90.02,36.71,135.0,-0.43,0.94,27.82,XR,NTT,BANKS
67,SONACOMS,1006.0,-39.20,41.0,M-SC,11.99,75705.0,-25454.0,100347.0,-1.67,-25.16,132.55,74.03,202.0,-0.25,0.55,6.75,AR,BTT,AUTO


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,100.70,62.0,H-SC,12.58,126603.0,-15876.0,35398.0,-0.74,-11.14,27.96,13.70,163.0,-0.45,0.92,49.43,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,103.84,58.0,M-SC,4.99,87849.0,-12928.0,13019.0,-0.38,-12.83,14.82,0.09,245.0,-0.99,0.64,15.41,OX40N,NTT,DURABLES
44,KANSAINER,340.0,-67.42,60.0,H-SC,2.00,224100.0,-45567.0,81909.0,0.97,-16.90,36.55,13.47,138.0,-0.56,1.63,13.35,XY24,NTT,PAINTS
19,CERA,9475.0,-19.81,46.0,H-SC,2.14,146038.0,-29865.0,71880.0,0.13,-16.98,49.22,23.89,149.0,-0.42,1.06,26.12,OX40N,NTT,CERAMICS
72,TATAELXSI,9161.0,-20.18,52.0,H-MC,7.98,107930.0,-23905.0,66129.0,-1.25,-18.13,61.27,32.03,98.0,-0.36,0.79,20.89,OX40N,NTT,IT


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4998.00,-26.12,55.0,H-LC,13.39,294784.0,-51172.0,175043.0,0.38,-14.79,59.38,35.80,1.0,-0.29,2.15,4.43,X40,BTT,IT
39,INFY,2275.00,-16.37,56.0,H-LC,9.45,325698.0,12810.0,158875.0,1.28,4.09,48.78,54.87,3.0,0.08,2.37,11.02,X40,BTT,IT
41,ITC,452.00,-37.08,56.0,H-LC,2.98,202837.0,2699.0,18194.0,-0.06,1.35,8.97,10.44,4.0,0.15,1.48,6.99,X40,NTT,FMCG
83,VBL,671.64,-16.73,37.0,H-LC,6.05,299796.0,-16046.0,128702.0,-1.00,-5.08,42.93,35.67,5.0,-0.12,2.18,8.07,X40N,ATH,FMCG
1,ABB,7934.00,-40.06,53.0,H-LC,11.97,250320.0,-11299.0,130517.0,-0.06,-4.32,52.14,45.57,7.0,-0.09,1.82,6.51,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.00,44.35,64.0,M-MC,6.95,227287.0,2325.0,165329.0,0.13,1.03,72.74,74.53,192.0,0.01,1.66,33.62,XY24,BTT,STEEL
15,BLUESTARCO,2326.38,9.48,53.0,H-MC,3.96,189070.0,24400.0,43562.0,0.21,14.82,23.04,41.28,89.0,0.56,1.38,23.98,X40N,ATH,AC
84,VOLTAS,1918.49,0.04,56.0,H-MC,2.97,210000.0,18258.0,77763.0,0.34,9.52,37.03,50.08,99.0,0.23,1.53,16.97,XY25,ATH,AC
24,DABUR,735.00,-3.84,55.0,H-MC,6.15,209488.0,12460.0,77154.0,-1.28,6.32,36.83,45.49,102.0,0.16,1.53,20.19,XY24,BTT,FMCG
18,CAMS,5211.76,-5.61,49.0,H-SC,4.76,107380.0,5376.0,38549.0,-0.40,5.27,35.90,43.06,122.0,0.14,0.78,23.29,X40N,ATH,MISC


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,NATIONALUM,247.44,-42.83,71.0,H-MC,3.48,113632.0,13128.0,17511.0,2.52,13.06,15.41,30.49,79.0,0.75,0.83,49.70,MH,ATH,METALS
85,WHIRLPOOL,2270.00,-40.03,55.0,M-SC,1.41,101258.0,9760.0,68997.0,1.51,10.67,68.14,86.07,223.0,0.14,0.74,47.04,XR,NTT,DURABLES
36,INDIAMART,4850.92,-50.35,56.0,H-SC,12.28,138812.0,15476.0,118282.0,0.26,12.55,85.21,108.45,119.0,0.13,1.01,37.71,AR,ATH,MISC
86,WIPRO,420.00,-11.07,54.0,M-LC,5.81,156178.0,5233.0,104218.0,-0.79,3.47,66.73,72.51,53.0,0.05,1.14,9.70,XR,NTT,IT
12,BANKINDIA,190.00,-30.43,61.0,H-MC,5.62,180561.0,753.0,111659.0,-0.59,0.42,61.84,62.52,88.0,0.01,1.32,34.65,XR,NTT,BANKS


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-41.65,55.0,L-SC,44.51,86784.0,-26765.0,66815.0,-1.03,-23.57,76.99,35.27,268.0,-0.40,0.63,112.41,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-5.85,43.0,H-SC,14.00,87673.0,-13290.0,107820.0,-0.54,-13.16,122.98,93.63,148.0,-0.12,0.64,30.59,SR,ATH,CHEMICALS
48,MASFIN,398.61,-15.99,59.0,H-SC,13.15,95685.0,-2295.0,23902.0,0.97,-2.34,24.98,22.05,152.0,-0.10,0.70,39.09,XR,ATH,FINANCE
49,NATIONALUM,247.44,-42.83,71.0,H-MC,3.48,113632.0,13128.0,17511.0,2.52,13.06,15.41,30.49,79.0,0.75,0.83,49.70,MH,ATH,METALS
82,VALIANTORG,838.00,-324.58,54.0,H-SC,7.79,124865.0,-38740.0,160052.0,0.62,-23.68,128.18,74.15,139.0,-0.24,0.91,60.06,XR,NTT,CHEMICALS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,ATULAUTO,844.00,3570.27,56.0,M-SC,7.49,114366.0,-33002.0,84814.0,-2.39,-22.39,74.16,35.16,236.0,-0.39,0.83,16.81,XY24,NTT,AUTO
22,COFFEEDAY,80.00,-41.65,55.0,L-SC,44.51,86784.0,-26765.0,66815.0,-1.03,-23.57,76.99,35.27,268.0,-0.40,0.63,112.41,XR,NTT,HOTELS
49,NATIONALUM,247.44,-42.83,71.0,H-MC,3.48,113632.0,13128.0,17511.0,2.52,13.06,15.41,30.49,79.0,0.75,0.83,49.70,MH,ATH,METALS
27,GREENPANEL,537.00,246.25,73.0,M-SC,5.36,154696.0,-28382.0,107359.0,-1.60,-15.50,69.40,43.14,230.0,-0.26,1.13,45.51,XY24,NTT,MISC
75,TITAGARH,1548.00,-2.32,65.0,H-SC,5.82,160300.0,-31715.0,110607.0,1.68,-16.52,69.00,41.08,158.0,-0.29,1.17,35.83,XY24,NTT,ENGINEERING


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.77
1,25,43.52
2,50,73.94


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    42.45
LC    34.54
MC    23.02
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.92
X40      15.31
X40N     12.96
XR       10.03
XY25      9.77
AR        8.62
OX40N     8.21
X200      1.74
MH        1.72
X5K       1.45
SR        1.28
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    27.39
H-LC    26.30
H-MC    20.07
M-SC    13.52
M-LC     7.18
M-MC     2.64
L-SC     1.54
L-LC     1.06
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,13.42,-14.16,79.07
FMCG,11.84,-4.08,40.50
FINANCE,9.62,-16.68,62.38
BANKS,7.80,-12.13,63.75
MISC,6.10,-13.48,78.27
PAINTS,5.92,-13.55,38.97
ELECTRICAL,5.59,-7.30,66.43
AC,3.62,2.43,30.61
INSURANCE,3.40,-7.95,44.97


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2874171.0,22
AR,1314687.0,9
XR,1237084.0,13
X40,865772.0,10
X40N,633037.0,10
OX40N,520316.0,10
XY25,410340.0,7
SR,245873.0,2
X5K,118017.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3357630.0,28
M-SC,1390954.0,17
H-LC,1382642.0,15
H-MC,1198835.0,15
M-LC,392760.0,5
M-MC,315451.0,2
L-SC,248652.0,3
L-MC,60216.0,1
L-LC,38355.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1040988.0      6
M-SC       XY24       805037.0      7
H-SC       AR         783126.0      5
           XR         681059.0      6
H-LC       X40        577201.0      5
H-MC       XY24       487275.0      4
H-LC       AR         368588.0      2
M-MC       XY24       315451.0      2
H-SC       X40N       298014.0      4
           OX40N      245939.0      4
           SR         245873.0      2
H-LC       X40N       226112.0      4
H-MC       X40        218976.0      4
           XY25       188374.0      2
L-SC       XR         163831.0      2
M-SC       AR         162973.0      2
M-LC       XY24       159967.0      2
M-SC       OX40N      123427.0      4
M-LC       X5K        118017.0      1
M-SC       XR         116101.0      2
           XY25       113821.0      1
H-MC       XR         111659.0      1
           X40N       108911.0      2
M-LC       XR         104218.0      1
H-LC       X200        86056.0      1
L-SC       OX40N       84821.0      1
M-SC       X40         69595.0      1
H-MC       OX40N       66129.0      1
H-LC       XY24        65453.0      1
H-SC       MH          62631.0      1
L-MC       XR          60216.0      1
H-LC       XY25        59232.0      2
L-LC       XY25        38355.0      1
H-MC       MH          17511.0      1
M-LC       XY25        10558.0      1

In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 37.0 seconds
